In [1]:
import syft as sy
import torch as th
import copy
import random

In [2]:
hook = sy.TorchHook(th)
bob = sy.VirtualWorker(hook, id="bob")

In [3]:
def replace_ints(obj, change_id, to_id):
    for i, item in enumerate(obj):
        if isinstance(item, int):
            if(item == change_id):
                obj[i] = to_id
                
        if isinstance(item, bytes):
            if(item == b'plan'):
                obj[i] = b'bob'
            
        elif isinstance(item, (list, tuple)):
            replace_ints(item, change_id, to_id)

In [4]:
def create_plan(plan_blueprint, *args):
    
    args = list(args)
    
    plan = sy.workers.Plan(hook=hook, owner=sy.local_worker, id="plan")
    
    arg_ids = list()
    
    for i in range(len(args)):
        args[i] = args[i].send(plan)
        arg_ids.append(args[i].id_at_location)
    
    plan.arg_ids = arg_ids
    
    res_ptr = plan_blueprint(*args)
    
    plan.result_ids = [res_ptr.id_at_location]

    def my_plan(worker, *_args):
        
        plan2 = sy.workers.Plan(hook, id="plan2")
        plan2.readable_plan = copy.deepcopy(plan.readable_plan)
        plan2.plan = copy.deepcopy(plan.plan)
                
        # for every argument
        for i in range(len(_args)):
            # for every message
            for msg in plan2.readable_plan:
                # look for the old id and replace it with the new one
                replace_ints(msg, args[i].id_at_location, _args[i].id_at_location)
        
        plan2.execute_plan(on_worker=worker)
        
        return sy.PointerTensor(id=random.randint(0,10e10),
                                id_at_location=res_ptr.id_at_location,
                               location=worker,
                               owner=res_ptr.owner).wrap()
    
    return my_plan, plan

In [5]:
def plan_blueprint(data):
    return data.abs()

my_plan, plan = create_plan(plan_blueprint, th.tensor([-1,-2]))

My owner:<VirtualWorker id:me #tensors:0>


In [6]:
plan_ptr = sy.local_worker.send(obj=plan, workers=bob)

My owner:<VirtualWorker id:bob #tensors:0>


In [7]:
plan_ptr.location

<VirtualWorker id:bob #tensors:1>

In [8]:
response = plan_ptr(th.tensor([1,2,-3]).send(bob))

AttributeError: 'Plan' object has no attribute 'execute_cmd'

In [12]:
response.get()

tensor([1, 2, 3])

In [15]:
bob._objects

{b'plan': <Plan id:b'plan' #tensors:0>,
 46958365321: tensor([-1, -2,  3]),
 1234567: tensor([1, 2, 3])}

In [15]:
response.get()

tensor([1, 2, 3])

In [11]:
bob._objects

{b'plan': <Plan id:b'plan' #tensors:0>,
 43293274163: tensor([-1, -2,  3]),
 73990899615: tensor([1, 2, 3])}

In [11]:
bob._objects[b'plan'].owner

<VirtualWorker id:bob #tensors:3>

In [7]:
plan_ptr

<PlanPointer id:83472359010 #tensors:0>

In [7]:
bob._objects

{b'plan': <Plan id:b'plan' #tensors:0>}

In [6]:
plan_ptr = plan.create_pointer()

In [7]:
plan_ptr

<PlanPointer id:79274252196 #tensors:0>

In [9]:
splan = sy.serde.serialize(plan)

In [12]:
splan

b'0\x92\r\x92\x92\x03\x91\x92\x02\x92\x02\x92\x02\x92\x01\x92\x03\x92\x02\x92\x03\x92\x92\x03\x92\x02\x92\x03\x93\xa3abs\x92\x03\x92\x0b\x92\x03\x95\xcf\x00\x00\x00\x0f\xbaXi\xee\xcf\x00\x00\x00\x0fT\x07\x0eM\xa4plan\xc0\x92\x03\x91\x02\x92\x03\x92\x02\x92\x03\x90\xcf\x00\x00\x00\x12\xe1 A\x0c\xa4plan'

In [11]:
sy.serde.deserialize(splan)

<Plan id:b'plan' #tensors:0>

In [16]:
x = th.tensor([-2,-3]).send(bob)

In [17]:
result = my_plan(bob, x)

Execute Plan
(2, (1, [2, [[2, [b'abs', [11, [82930123604, 34993490673, b'bob', None, [2]]], [2, []]]], 41494258400]]))


In [8]:
result.get()

tensor([2, 3])

In [18]:
bob._objects

{61931654956: tensor([2, 3, 4]), 92080015097: tensor([-2, -3, -4])}

In [7]:
res_ptr

(Wrapper)>[PointerTensor | me:85843839224 -> bob:94076966911]

In [9]:
res = res_ptr.get()

assert (res == expected).all()

Execute Plan
(2, [0, [26355456353, b'\x80\x02\x8a\nl\xfc\x9cF\xf9 j\xa8P\x19.\x80\x02M\xe9\x03.\x80\x02}q\x00(X\x10\x00\x00\x00protocol_versionq\x01M\xe9\x03X\r\x00\x00\x00little_endianq\x02\x88X\n\x00\x00\x00type_sizesq\x03}q\x04(X\x05\x00\x00\x00shortq\x05K\x02X\x03\x00\x00\x00intq\x06K\x04X\x04\x00\x00\x00longq\x07K\x04uu.\x80\x02ctorch._utils\n_rebuild_tensor_v2\nq\x00((X\x07\x00\x00\x00storageq\x01ctorch\nFloatStorage\nq\x02X\x0f\x00\x00\x00140384401788496q\x03X\x03\x00\x00\x00cpuq\x04K\x03Ntq\x05QK\x00K\x03\x85q\x06K\x01\x85q\x07\x89ccollections\nOrderedDict\nq\x08)Rq\ttq\nRq\x0b.\x80\x02]q\x00X\x0f\x00\x00\x00140384401788496q\x01a.\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\xbf\x00\x00\x00@\x00\x00@@', None, None, None, None]])
(1, [2, [[2, [b'abs', [11, [72058580809, 26355456353, b'bob', None, [3]]], [2, []]]], 89437548570]])
(3, 89437548570)


In [9]:
res

tensor([1., 2., 3.])

In [4]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [5]:
y = x + x

In [7]:
y.get()

tensor([ 2,  4,  6,  8, 10])